In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Get current Directory
wd = os.getcwd()
# Add package to path
sys.path.append(os.path.join(os.path.abspath(os.path.join(wd, os.pardir)), 'IGTLinkPython'))

In [ ]:
import OpenIGTLink
import numpy

## Create Header for IGTLink Message

In [ ]:
header = OpenIGTLink.OpenIGTLinkHeader()
header.setTYPE('STATUS')
header.setDEVICE_NAME('Device Name Example')
header.setTIME_STAMP(123,0.456)

bodyExample = 'This is an example of body message'
header.setCRC64(bodyExample) # Need the body to calculate the CRC
header.setBODY_SIZE(len(bodyExample))

print header

messageHeader = header.getHeaderMessage()
headerTest = OpenIGTLink.OpenIGTLinkHeader()
headerTest.unpack(messageHeader)
headerTest.setCRC64(bodyExample)
print headerTest

## Create Body for IGTLink Message

In [ ]:
body = OpenIGTLink.OpenIGTLinkBody()
bodyExample = 'This is an example of body message'
body.setBodyMessage(bodyExample) # Need the body to calculate the CRC
print body

## Create IGTLink Message

In [ ]:
igtmessage = OpenIGTLink.OpenIGTLinkMessage()
igtmessage.header = OpenIGTLink.OpenIGTLinkHeader()
igtmessage.header.setTYPE('STATUS')
igtmessage.header.setDEVICE_NAME('Device Name Example')
igtmessage.header.setTIME_STAMP(123,0.456)
bodyExample = 'This is an example of body message'
igtmessage.body.setBodyMessage(bodyExample)
bodyCoding = '>' + 'c' * len(bodyExample)
igtmessage.__updateMessage__(bodyCoding)
print igtmessage

## Create IGTLink TRANSFORM

In [ ]:
transform = OpenIGTLink.OpenIGTLinkTransform()

npTransform = numpy.eye(4)
npTransform[0,0] = 2
npTransform[1,1] = 3
npTransform[3,1] = 15

message = transform.setOpenIGTLinkTransform( npTransform = npTransform,\
                                            floatTimeStamp = 123.456,\
                                            transformName = 'RigidToTracker Test')
print transform

transformTest = OpenIGTLink.OpenIGTLinkTransform()
transformTest.header.unpack(message[0:transformTest.header.IGTLinkHeaderSize])
transformTest.unpackTransform(message[transformTest.header.IGTLinkHeaderSize::])
print transformTest

## Create IGTLink STATUS

In [ ]:
status = OpenIGTLink.OpenIGTLinkStatus()
message = status.setOpenIGTLinkStatus(statusCode = 9,\
                                         statusSubCode = 254,\
                                         errorName = 'Example Error Name',\
                                         statusMessage = None,\
                                         deviceName = 'Tracker',\
                                         floatTimeStamp = 123.456)
print status

statusTest = OpenIGTLink.OpenIGTLinkStatus()
statusTest.header.unpack(message[0:transformTest.header.IGTLinkHeaderSize])
statusTest.unpackStatus(message[transformTest.header.IGTLinkHeaderSize::])
print statusTest

## Read Real Messages and decode

In [ ]:
import struct
messagefile = os.path.join(os.path.join(os.path.join(os.path.abspath(os.path.join(wd, os.pardir)),\
                                             'IGTLinkPython'),\
                                             'ExampleFiles'),\
                                             'ExampleMessages.txt')

text_file = open(messagefile, "r")
messages = text_file.read()
text_file.close()

firstMessage = messages[0:106]
a =  repr(firstMessage)

transform = OpenIGTLink.OpenIGTLinkTransform()
transform.header.unpack(firstMessage[0:transform.header.IGTLinkHeaderSize])
transform.unpackTransform(firstMessage[transform.header.IGTLinkHeaderSize::])
print transform

## Read Message File

In [ ]:
import pandas
messagefile = os.path.join(os.path.join(os.path.join(os.path.abspath(os.path.join(wd, os.pardir)),\
                                             'IGTLinkPython'),\
                                             'ExampleFiles'),\
                                             'Model.csv')
df = pandas.read_csv(messagefile)
df

In [ ]:
messageList = list()
for i in range(len(df.index)):
    message = df.loc[i]
    
    m = None
    if message['TYPE'] == 'TRANSFORM':
        
        npTransform = numpy.eye(4)
        for ii in range(3):
            for jj in range(4):
                npTransform[ii,jj] = float(message['T' + str(ii) + str(jj)])
        
        
        t = OpenIGTLink.OpenIGTLinkTransform()
        m = t.setOpenIGTLinkTransform(npTransform = npTransform,\
            floatTimeStamp = float(message['TIME_STAMP']),\
            transformName = message['DEVICE_NAME'])
        messageList.append(m)
    
    elif message['TYPE'] == 'STATUS':
        s = OpenIGTLink.OpenIGTLinkStatus()
        m = s.setOpenIGTLinkStatus(statusCode = int(message['CODE']),\
                                         statusSubCode = int(message['SUBCODE']),\
                                         errorName = message['ERROR_NAME'],\
                                         statusMessage = message['STATUS_MESSAGE'],\
                                         deviceName = message['DEVICE_NAME'],\
                                         floatTimeStamp = float(message['TIME_STAMP']))
        messageList.append(m)
    else:
        print "Problem realind row: ", i
    
print messageList

## Create and launch server

In [ ]:
server = OpenIGTLink.OpenIGTLinkServer()
server.loadFileMessages(messagefile)
server.connect()
server.startSending(10)

In [ ]:
import helpers
helpers.PrintPercentage(52,'Hey')

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Get current Directory
wd = os.getcwd()
# Add package to path
sys.path.append(os.path.join(os.path.abspath(os.path.join(wd, os.pardir)), 'IGTLinkPython'))
import OpenIGTLink

messagefile = os.path.join(os.path.join(os.path.join(os.path.abspath(os.path.join(wd, os.pardir)),\
                                             'IGTLinkPython'),\
                                             'ExampleFiles'),\
                                             'Model.csv')

server = OpenIGTLink.OpenIGTLinkServer()
server.loadFileMessages(messagefile)
server.connect()
server.startSending(10)